<a href="https://colab.research.google.com/github/adityajhakumar/WALMART_ai_CHATBOT/blob/main/ai%20chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [27]:
!pip install gtts
!pip install IPython


In [30]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import io
import threading
import time

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2LMHeadModel.from_pretrained('gpt2-large', pad_token_id=tokenizer.eos_token_id)

# Load Excel data
food_recipes = pd.read_excel('/content/food_recipes_large.xlsx')
supermarket_items = pd.read_excel('/content/supermarket_items_large.xlsx')

# Clean column names by stripping any extra spaces
food_recipes.columns = food_recipes.columns.str.strip()
supermarket_items.columns = supermarket_items.columns.str.strip()

# Initialize memory for shopping preferences
shopping_preferences = {}

def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model.generate(inputs, max_length=100, num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

def get_random_examples(df, num_examples=5):
    return df.sample(n=num_examples)

def get_supermarket_info(item_name):
    item_info = supermarket_items[supermarket_items['Item Name'].str.contains(item_name, case=False, na=False)]
    if not item_info.empty:
        return item_info[['Item Name', 'Price', 'Brand']].to_dict(orient='records')
    return None

def remember_shopping_preference(item_name, category):
    if category not in shopping_preferences:
        shopping_preferences[category] = []
    shopping_preferences[category].append(item_name)

def handle_shopping_query(item_name):
    supermarket_info = get_supermarket_info(item_name)
    if supermarket_info:
        response = f"You can buy {item_name} at Walmart. Here are the details:\n"
        for item in supermarket_info:
            response += f"{item['Item Name']} - Brand: {item['Brand']}, Price: ${item['Price']}\n"
        remember_shopping_preference(item_name, 'Shopping')
    else:
        response = f"Sorry, I couldn't find {item_name} in the supermarket database. Let me see if I can find something else for you."
        response += generate_response(f"Can you help me find {item_name}?")
    return response

def initial_welcome():
    food_examples = get_random_examples(food_recipes, 5)
    supermarket_examples = get_random_examples(supermarket_items, 5)

    food_examples_str = ', '.join(food_examples['Food Item Name'])
    supermarket_examples_str = ', '.join(supermarket_examples['Item Name'])

    response = (
        "Hello! I'm here to help you with your shopping. What type of items are you interested in today?\n\n"
        "Here are some examples of food items you might be interested in:\n"
        f"{food_examples_str}\n\n"
        "And here are some items available at the supermarket:\n"
        f"{supermarket_examples_str}\n"
    )
    return response

def show_more_examples(df, num_examples=10):
    return df.sample(n=num_examples)

# Start conversation loop
print("Chatbot: Hello! I'm here to help you with your shopping. What type of items are you interested in today?")
response = initial_welcome()
print("Chatbot:", response)

while True:
    print("Waiting for user input...")
    user_input = input("You: ")
    print(f"Received user input: {user_input}")
    if user_input.lower() in ["exit", "quit", "bye"]:
        response = "Goodbye! Have a great day!"
        print(f"Chatbot: {response}")
        break

    if "show me more" in user_input.lower():
        category = "food" if "food" in user_input.lower() else "supermarket"
        examples = show_more_examples(food_recipes if category == "food" else supermarket_items)
        examples_str = ', '.join(examples[examples.columns[1]])
        response = f"Here are more examples for you:\n{examples_str}"
    elif "buy" in user_input.lower() or "walmart" in user_input.lower():
        item_name = user_input.lower().replace("i want to buy", "").replace("from walmart", "").strip()
        response = handle_shopping_query(item_name)
    else:
        response = generate_response(user_input)

    print("Chatbot:", response)
    time.sleep(2)  # Add a delay to reduce request frequency


Chatbot: Hello! I'm here to help you with your shopping. What type of items are you interested in today?
Chatbot: Hello! I'm here to help you with your shopping. What type of items are you interested in today?

Here are some examples of food items you might be interested in:
Chocolate Cake, Chicken Salad, Chicken Curry, Spaghetti Carbonara, Chicken Salad

And here are some items available at the supermarket:
Classic Black Dress, Whole Milk, Cooking Oil, Classic Black Dress, Chicken Wings

Waiting for user input...
You: Chicken Salad
Received user input: Chicken Salad
Chatbot: Chicken Salad

1/2 cup shredded cheddar cheese, plus more for garnish (I used a mix of mozzarella and parmesan, but you can use whatever cheese you like)


2 tablespoons chopped fresh parsley, or more if you'd like it a little spicier (optional) Instructions Preheat the oven to 400 degrees F. Spray a baking sheet with nonstick cooking spray and set aside. In a large bowl, whisk together the flour, baking powder
Wa